In [1]:
import os
import json
import pandas as pd
import traceback

In [2]:
from langchain.chat_models import ChatOpenAI

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
KEY=os.getenv("OPENAI_API_KEY")

In [5]:
llm=ChatOpenAI(openai_api_key=KEY,model_name="gpt-3.5-turbo", temperature=0.5)

e:\Turja\LLM's\MCQ-Generator\mcq\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [7]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [8]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [9]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [10]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [11]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [12]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""


In [13]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [14]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [15]:
generate_evaluate_chain = SequentialChain(
    chains=[quiz_chain, review_chain],
    input_variables=["text", "number", "subject", "tone", "response_json"],
    output_variables=["quiz", "review"],
    verbose=True,
)

In [16]:
data = r"E:\Turja\LLM's\MCQ-Generator\data.txt"
data

"E:\\Turja\\LLM's\\MCQ-Generator\\data.txt"

In [17]:
with open(data, 'r') as file:
    TEXT = file.read()

In [18]:
print(TEXT)

Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying
themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that 
process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is 
evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it 
allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal 
environments.[1][2][3][4][5] Biologists are able to study life at multiple levels of organization,[1] from the molecular 
biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there 
are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that 
they use.[7][8][9] Like other scientist

In [19]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [20]:
NUMBER=5 
SUBJECT="biology"
TONE="simple"

In [21]:
with get_openai_callback() as cb:
    response = generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject": SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON),
        }
    )

e:\Turja\LLM's\MCQ-Generator\mcq\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying
themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that 
process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is 
evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it 
allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal 
environments.[1][2][3][4][5] Biologists are able to study life at multiple levels of organization,[1] from the molecular 
biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there 
are multiple subdisciplines within biology, each d

In [22]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:1963
Prompt Tokens:1228
Completion Tokens:735
Total Cost:0.003312


In [23]:
response

{'text': 'Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying\nthemes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that \nprocess hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is \nevolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it \nallows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal \nenvironments.[1][2][3][4][5] Biologists are able to study life at multiple levels of organization,[1] from the molecular \nbiology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there \nare multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that \nthey use.[7][8][9] Li

In [24]:
quiz=response.get("quiz")

In [25]:
quiz=json.loads(quiz)

In [26]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
        ]
    )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [27]:
quiz_table_data

[{'MCQ': 'What is the scientific study of life?',
  'Choices': 'a: Chemistry | b: Biology | c: Physics | d: Geology',
  'Correct': 'b'},
 {'MCQ': 'Which theme in biology explains the unity and diversity of life?',
  'Choices': 'a: Evolution | b: Energy processing | c: Cell structure | d: Genetic information',
  'Correct': 'a'},
 {'MCQ': 'What allows organisms to move, grow, and reproduce?',
  'Choices': 'a: Photosynthesis | b: Energy processing | c: Cell division | d: Respiration',
  'Correct': 'b'},
 {'MCQ': 'What is the process by which biologists make observations, pose questions, and form conclusions?',
  'Choices': 'a: Scientific method | b: Guesswork | c: Intuition | d: Magic',
  'Correct': 'a'},
 {'MCQ': 'Which organisms contribute to the biodiversity of an ecosystem?',
  'Choices': 'a: Humans only | b: Plants only | c: All organisms | d: Animals only',
  'Correct': 'c'}]

In [28]:
quiz=pd.DataFrame(quiz_table_data)

In [29]:
quiz

,MCQ,Choices,Correct
0,What is the scientific study of life?,a: Chemistry | b: Biology | c: Physics | d: Ge...,b
1,Which theme in biology explains the unity and ...,a: Evolution | b: Energy processing | c: Cell ...,a
2,"What allows organisms to move, grow, and repro...",a: Photosynthesis | b: Energy processing | c: ...,b
3,What is the process by which biologists make o...,a: Scientific method | b: Guesswork | c: Intui...,a
4,Which organisms contribute to the biodiversity...,a: Humans only | b: Plants only | c: All organ...,c


In [30]:
quiz.to_csv("biologyquiz.csv",index=False)